In [12]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
from datetime import datetime
from io import BytesIO
from PIL import Image
import boto3
from selenium.webdriver.common.by import By



In [13]:

# 페이지 크롤링 함수
def cardCrawling (cardurl):
    benefits=""
    
    cardhtml=urlopen(cardurl)
    cardbs=BeautifulSoup(cardhtml,'html.parser')
    
    # benefits
    benes = cardbs.findAll('div',{'class','bnfCont'})
    if len(benes) == 0:
        beneLists = cardbs.find('ul',{'class','toggleList'}).findAll('li',recursive=False)
        for beneList in beneLists:
            titlebene = beneList.find('a').text.replace('\t','').replace('\r','').replace('\n','').replace(' ','')
            if titlebene=="L.POINT" or titlebene=="가족카드" or titlebene== "가족카드안내" or titlebene== "연회비": continue
            benefits+="<"+titlebene+"> "
            details = beneList.find('div',{'class','toggleCont'}).findAll(recursive=False)
            for detail in details:
                if str(detail)[1:6]=="table": 
                    benefits+=str(detail).replace('\n','')
                elif str(detail)[1:3]=="h3": 
                    benefits+="["+detail.text+"]"
                elif str(detail)[1:3]=="h4": 
                    benefits+="/"+detail.text+": "
                elif str(detail)[1:6]=="style": 
                    continue
                else:
                    if detail.find('table') is not None:
                        benefits+=str(detail).replace('\n','')
                    else:
                        benefits+=detail.text.replace('\n','')
                benefits.replace('\n','').replace('\t','')
            benefits+="\n"
    else:
        for bene in benes:
            titlebene=bene.find('h3').text
            if titlebene=="L.POINT" or titlebene=="가족카드" or titlebene== "가족카드 안내" or titlebene== "연회비" or titlebene== "혜택 모아보기": continue
            benefits+="<"+titlebene+">"
            sections = bene.findAll('div',{'class','toggle'})
            for section in sections:
                beneNames=section.find('h4')
                beneNameText=beneNames.text
                benefits+="["+beneNameText+"] "
                details = section.find('div',{'class','toggleCont'}).findAll(recursive=False)

                for detail in details:
                    # print(detail)
                    # print()
                    if str(detail)[1:6]=="table": 
                        benefits+=str(detail).replace('\n','')
                    elif str(detail)[1:3]=="h3": 
                        benefits+="["+detail.text+"]"
                    elif str(detail)[1:3]=="h4": 
                        benefits+="/"+detail.text+": "
                    elif str(detail)[1:6]=="style": 
                        continue
                    else:
                        if detail.find('table') is not None:
                            benefits+=str(detail).replace('\n','')
                        else:
                            benefits+=detail.text.replace('\n','')
                benefits.replace('\n','').replace('\r','').replace('\t','')
            benefits+="\n"
            benefits=benefits.replace("'","")
            
    return benefits 


In [15]:

def cardList(associate):
    url = 'https://www.lottecard.co.kr/app/LPCDAEA_V100.lc'
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)

    # 웹 페이지 로드
    driver.get(url)

    if(associate):
        # 제휴 카드 클릭
        link = driver.find_element(By.LINK_TEXT,"제휴")
        link.click()
        time.sleep(5)

    # Selenium으로 페이지 스크랩
    rendered_html = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(rendered_html, 'html.parser')

    # 더보기 버튼 끝까지 누르기
    while True:
        new_render_html=driver.page_source
        soup = BeautifulSoup(new_render_html, 'html.parser')
        if soup.find('button',{'id':'btnMore'}) is None: break
        driver.find_element("id","btnMore").click()
        time.sleep(3)

    return soup.find('ul', {'id':'ajaxCardList'}).findAll('li')



In [16]:

cardlist=cardList(False)

In [17]:

name = []
img_url = []
benefits = []
created_at = []

for card in cardlist:
    cardNo = card.find('a').get('onclick')
    cardNo = re.search(r"'(.*?)'", cardNo).group(1)
    
    cardurl='https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC='+cardNo
    
    cardImg= "https:" + card.find('img').get('src')
    img_url.append("https://once-s3.s3.ap-northeast-2.amazonaws.com/lottecard/"+cardNo+".png")
    
    cardName=card.find('b').text
    name.append(cardName)

    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S")+" ["+cardName+"] --- 웹 페이지에 접속 중... ")
    
    benefit = cardCrawling(cardurl)
    benefits.append(benefit)

    now_datetime = datetime.now()
    formatted_now = now_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")
    created_at.append(formatted_now)

2024-04-09 07:27:53 [롯데체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:53 [롯데포인트 플러스 체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:54 [I♥Busan 체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:54 [롯데 비즈니스 체크 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:54 [SUPER PLUS 체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:55 [롯데체크카드VISA] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:55 [롯데 체크플러스 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:55 [위클리 체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:55 [위클리 VISA 롯데체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:56 [롯데 체크카드 은련(UPI)] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:56 [롯데 포인트플러스 체크카드 은련 (UPI)] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:56 [위클리 체크카드 은련] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:57 [LIKIT fun 체크플러스 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:57 [LIKIT fun 체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:57 [롯데체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:58 [LIKIT on 체크플러스 카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:58 [LIKIT on 체크카드] --- 웹 페이지에 접속 중... 
2024-04-09 07:27:58 [LIKIT all 체크플러스 카드] --- 웹 페

In [18]:
cardlist2=cardList(True)

In [19]:
for card in cardlist2:
    cardNo = card.find('a').get('onclick')
    cardNo = re.search(r"'(.*?)'", cardNo).group(1)
    
    cardurl='https://www.lottecard.co.kr/app/LPCDADB_V100.lc?vtCdKndC='+cardNo
    
    cardImg= "https:" + card.find('img').get('src')
    img_url.append("https://once-s3.s3.ap-northeast-2.amazonaws.com/lottecard/"+cardNo+".png")
    
    cardName=card.find('b').text
    name.append(cardName)

    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S")+" ["+cardName+"](제휴) --- 웹 페이지에 접속 중... ")
    
    benefit = cardCrawling(cardurl)
    benefits.append(benefit)

    now_datetime = datetime.now()
    formatted_now = now_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")
    created_at.append(formatted_now)

2024-04-09 07:28:19 [세븐일레븐 멤버십롯데체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:19 [쉐보레오토 체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:19 [SKYPASS롯데 플래티넘 체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:20 [SK롯데 체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:20 [해병대 전우 롯데 포인트플러스 체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:20 [해병대 전우 롯데체크플러스 카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:21 [뉴롯데시네마 롯데체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:21 [쇼퍼홀릭 롯데홈쇼핑 롯데체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:21 [AIA생명 롯데 리얼체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:22 [경차 smart 롯데체크카드](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:22 [롯데 weekly 체크카드(경기도 청소년 교통비 지원)](제휴) --- 웹 페이지에 접속 중... 
2024-04-09 07:28:22 [E1 LPG 롯데 체크카드](제휴) --- 웹 페이지에 접속 중... 


In [20]:
card_company_id = [5] * len(name)
type = ["DebitCard"] * len(name)

data = {"card_company_id":card_company_id, "name" : name, "img_url" : img_url, "benefits": benefits, "created_at": created_at,"type":type}
df = pd.DataFrame(data)

df.to_csv("./debit_benefit.csv", encoding = "utf-8-sig", index=False)
print(datetime.now().strftime("%Y-%m-%d %H:%M:%S")+" 롯데카드 체크카드 크롤링 완료", flush=True)

2024-04-09 07:28:23 롯데카드 체크카드 크롤링 완료
